In [ ]:
!pip install finance-datareader

# 제출용 (content에 dacon_submit_api넣어야한다 )
# !pip install dacon_submit_api-0.0.4-py3-none-any.whl
# from dacon_submit_api import dacon_submit_api 

Processing ./dacon_submit_api-0.0.4-py3-none-any.whl


In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [ ]:
import pandas as pd
import os
import FinanceDataReader as fdr
import tqdm

import pandas as pd
import numpy as np
import pickle
import os
from datetime import datetime, timedelta

# [Regression]
from sklearn.linear_model import LinearRegression, Ridge

# [머신러닝 기법]
# bagging
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

# boosting
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

# [deep learning] 
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Dense, Activation, Flatten, Dropout
from keras.layers import LSTM, SimpleRNN, GRU

from sklearn.metrics import mean_absolute_error

In [ ]:
path="/gdrive/My Drive/Colab Notebooks/dacon/stock"
list_name = 'stock_list.csv'
stock_list = pd.read_csv(os.path.join(path,list_name))
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x : str(x).zfill(6))
display(stock_list)

stock_code = sorted(list(stock_list["종목코드"]))
print(stock_code)

,종목명,종목코드,상장시장
0,삼성전자,005930,KOSPI
1,SK하이닉스,000660,KOSPI
2,NAVER,035420,KOSPI
3,카카오,035720,KOSPI
4,삼성바이오로직스,207940,KOSPI
...,...,...,...
365,맘스터치,220630,KOSDAQ
366,다날,064260,KOSDAQ
367,제이시스메디칼,287410,KOSDAQ
368,크리스에프앤씨,110790,KOSDAQ


['000060', '000080', '000100', '000120', '000150', '000240', '000250', '000270', '000660', '000670', '000720', '000810', '000880', '000990', '001230', '001440', '001450', '001740', '002380', '002790', '003000', '003090', '003380', '003410', '003490', '003670', '003800', '004000', '004020', '004170', '004370', '004490', '004800', '004990', '005250', '005290', '005300', '005380', '005385', '005387', '005490', '005830', '005850', '005930', '005935', '005940', '006260', '006280', '006360', '006400', '006650', '006730', '006800', '007070', '007310', '007390', '008560', '008770', '008930', '009150', '009240', '009540', '009830', '009900', '010060', '010120', '010130', '010620', '010780', '010950', '011000', '011070', '011170', '011200', '011210', '011780', '012330', '012450', '012510', '012750', '013120', '013890', '014680', '015750', '015760', '016360', '016380', '017670', '017800', '018260', '018880', '019170', '020150', '021240', '022100', '023530', '024110', '025900', '025980', '026960',

In [ ]:
# sample_submission
path="/gdrive/My Drive/Colab Notebooks/dacon/stock"
list_name = 'sample_submission.csv'
sample_submission = pd.read_csv(os.path.join(path,list_name))
display(sample_submission)

my_sub = sample_submission.copy()

public_date = list(sample_submission["Day"][:5])
private_date = list(sample_submission["Day"][5:])
print(private_date)

,Day,000060,000080,000100,000120,000150,000240,000250,000270,000660,000670,000720,000810,000880,000990,001230,001440,001450,001740,002380,002790,003000,003090,003380,003410,003490,003670,003800,004000,004020,004170,004370,004490,004800,004990,005250,005290,005300,005380,005385,...,272290,273130,278280,278530,282330,285130,287410,290510,290650,292150,293490,293780,294090,294870,298000,298020,298050,298380,299030,299660,299900,307950,314130,316140,319400,319660,321550,323990,326030,330590,330860,336260,336370,347860,348150,348210,352820,357780,363280,950130
0,2021-11-01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2021-11-02,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2021-11-03,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2021-11-04,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2021-11-05,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,2021-11-29,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,2021-11-30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,2021-12-01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,2021-12-02,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,2021-12-03,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


['2021-11-29', '2021-11-30', '2021-12-01', '2021-12-02', '2021-12-03']


In [ ]:
# 주식장 끝난 후 아래 두줄만 수정하여 학습 후 제출한다. 26일 장끝난 후 마지막 학습!
train_start = datetime(2020,11,1) # train_start ~ val_start-1 => train
val_start = datetime(2021,11,16) # val_start ~ today => val 


test_start = datetime.today() + timedelta(days=1)

train_start_str = train_start.strftime("%Y%m%d")
val_start_str = val_start.strftime("%Y%m%d")
test_start_str = datetime.today().strftime("%Y%m%d")


model_names = ["XGBRegressor","LGBMRegressor","DecisionTreeRegressor","RandomForestRegressor","Ridge","LinearRegression"]

In [ ]:
def one_step_forward(_start,_end,_code):
    '''
    code에 해당하는 종목을 sequence=7로 데이터를 전처리한다 
    '''
    stock = fdr.DataReader(_code, start = _start, end = _end)
    stock_close = stock[["Close"]]
    stock_size = len(stock_close)

    x = []
    y = []
    sequence = 7

    for i in range(stock_size-sequence):
        x.append(list(stock_close["Close"][i:i+sequence]))
        y.append(stock_close["Close"][i+sequence])

    return x,y

def model_predict(x,y,model_name,start_day,pred_size):
    '''
    model_name 모델로 x,y를 학습하고, 향후 pred_size일을 예측하여 딕셔너리로 반환한다. 
    '''
    # train 
    if model_name=="XGBRegressor":
        model = XGBRegressor(learning_rate=0.05, n_estimators=100, random_state=123).fit(x, y)
    elif model_name=="LGBMRegressor":
        model = LGBMRegressor(learning_rate=0.05, n_estimators=100, random_state=123).fit(x, y)
    elif model_name=="DecisionTreeRegressor":
        model = DecisionTreeRegressor().fit(x, y)
    elif model_name=="RandomForestRegressor":
        model = RandomForestRegressor().fit(x, y)
    elif model_name=="Ridge":
        model = Ridge().fit(x, y)
    elif model_name=="LinearRegression":
        model = LinearRegression().fit(x, y)

    # predict
    pred_list = []
    x_pred=[x[-1][1:] + [y[-1]]]
    for i in range(pred_size):
        y_pred = model.predict(x_pred)[0]
        x_pred=[x_pred[0][1:]+[y_pred]]
        pred_list.append(y_pred)
    
    # 날짜
    pred_days = []
    d = start_day
    cnt = 0
    while cnt<pred_size:
        if d.weekday()>=5:
            d = d+timedelta(days=1)
        else:
            pred_days.append(d.strftime("%Y-%m-%d"))
            d = d+timedelta(days=1)
            cnt+=1

    return (pred_list,pred_days)

In [ ]:
for code in stock_code:
    train_x, train_y = one_step_forward(train_start_str, (val_start-timedelta(days=1)).strftime("%Y%m%d"), code) 
    val_y = list(fdr.DataReader(code, start = val_start_str, end = test_start.strftime("%Y%m%d"))["Close"])
    val_size = len(val_y)

    # val score확인후 best 모델 추출
    min_mae = 9876543210
    best_model=""
    for mn in model_names:
        val_y_hat = model_predict(train_x,train_y,mn,val_start,val_size)[0]
        now_mae = mean_absolute_error(val_y, val_y_hat)
        if now_mae<min_mae:
            min_mae=now_mae
            best_model=mn
    print(code, min_mae,best_model)

    # best모델을 이용해 피팅
    train_x, train_y = one_step_forward(train_start_str, test_start.strftime("%Y%m%d"), code)
    pred_list,pred_days = model_predict(train_x,train_y,best_model,test_start,6)
    
    pri_index = pred_days.index("2021-11-29")
    print(pred_list)
    print(pred_days)
    for i in range(5):
        my_sub[code][i+5]=float(pred_list[i+pri_index])

In [ ]:
my_sub

,Day,000060,000080,000100,000120,000150,000240,000250,000270,000660,000670,000720,000810,000880,000990,001230,001440,001450,001740,002380,002790,003000,003090,003380,003410,003490,003670,003800,004000,004020,004170,004370,004490,004800,004990,005250,005290,005300,005380,005385,...,272290,273130,278280,278530,282330,285130,287410,290510,290650,292150,293490,293780,294090,294870,298000,298020,298050,298380,299030,299660,299900,307950,314130,316140,319400,319660,321550,323990,326030,330590,330860,336260,336370,347860,348150,348210,352820,357780,363280,950130
0,2021-11-01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2021-11-02,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2021-11-03,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2021-11-04,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2021-11-05,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,2021-11-29,30776,30500,59304,133963,120343,15547,45808,80753,115500,642011,47686,207160,31642,72900,15119,1862,24538,5099,304700,44334,13064,30300,9890,7691,27283,159419,42777,80274,39808,221552,287270,81185,95800,32077,26400,36057,138562,205641,96827,...,39154,107593,346100,12803,153000,150948,6205,7194,33103,13587,101827,37937,51270,23111,263069,510000,634270,21483,51706,51543,42300,109166,35698,13288,3016,42862,18913,57477,98600,5659,44842,47969,96700,39350,20000,50413,377099,274434,25077,16634
6,2021-11-30,30612,32400,59304,130981,120343,15666,45808,80839,118000,639500,47686,206955,31527,72900,15195,1867,24608,5099,307771,44496,13081,30300,9735,7703,27369,158204,42791,80281,40547,221649,289470,81166,96400,32077,26400,36009,138441,209778,96922,...,39315,107348,346100,12922,153000,151416,6207,7272,33554,13499,102700,40400,51022,23197,263272,510000,646750,21386,51056,52059,46200,108768,35806,13363,3016,42596,18883,57924,97800,5674,45851,47869,103500,39700,20000,50481,384690,275245,24953,16634
7,2021-12-01,30231,32050,59304,130190,120343,15596,45808,80726,117500,639500,47686,208562,31514,72900,15420,1873,24788,5056,316315,44543,13064,30300,9812,7697,27437,158499,42742,80281,40913,221649,289280,79957,97100,32077,26400,35907,138887,208392,96817,...,39667,107411,348100,12813,158500,152066,6212,7194,33378,13440,102910,37400,51104,23294,262721,510000,644690,21402,50512,52098,50100,108768,35848,13405,3016,42479,18883,57924,98500,5677,46575,47708,112500,38150,20000,50595,384690,276625,24757,16696
8,2021-12-02,30231,32050,59304,130190,120343,15645,45808,80897,117500,639229,47686,208424,31394,72900,15456,1873,24933,5085,323470,44730,13064,30300,9771,7691,27423,157987,42801,80281,40973,221778,288765,80548,97000,32077,26400,35820,138967,208393,96922,...,39753,107341,348100,12802,165000,152048,6219,7194,33402,13443,103209,37100,51104,23195,264840,510000,643200,21411,50048,52374,40600,108768,35592,13334,3016,42426,18801,57924,98500,5674,46578,47708,98800,37200,20000,50465,384690,276557,24863,16639
9,2021-12-03,30231,31300,59304,130190,120343,15549,45808,80897,117500,639229,47686,208582,31855,72900,15551,1873,25236,5131,315713,44705,13081,30300,9782,7694,27501,156859,42856,80281,40717,221778,289600,80447,97300,32077,26400,35744,138903,210094,96922,...,39778,107250,345200,12783,177000,152131,6225,7194,34068,13435,103248,42700,51104,23036,265077,510000,637290,21419,49957,52867,42700,108768,35665,13243,3016,42504,18730,57924,98500,5664,46206,47905,96700,36950,20000,50521,384690,276838,24881,16639


In [ ]:
# 제출파일 만들기
def get_file_name():
    def returnNum(s):
        return int(s[:-4])

    files = os.listdir(path+"/sub")
    try:
        new_file_num = max(list(map(returnNum,files)))+1
        file_name = str(new_file_num)+".csv"
    except:
        file_name = "1.csv"
    return file_name 
    
file_name = get_file_name()
print(file_name)
my_sub.to_csv(path+"/sub/"+file_name,index=False)

7.csv
